In [1]:
import tkinter as tk
from tkinter import ttk
from tkinter import scrolledtext
from PIL import Image, ImageTk
import imdb
import webbrowser
import urllib.request
import io

ia = imdb.IMDb()

recommended_movies = []
current_index = 0

In [2]:
def search_button_click():
    keyword = keyword_entry.get().strip()
    if keyword:
        search_content(keyword)
    else:
        clear_results()
        result_text.insert(tk.END, "Please enter a movie name.")

def search_content(keyword):
    global recommended_movies, current_index
    # Clear previous results
    clear_results()
    # Search on IMDb
    movies = ia.search_movie(keyword)
    recommended_movies = movies[:5]  # Store the first 5 recommended movies
    current_index = 0
    # Display details of the first recommended movie
    display_movie_details(recommended_movies[current_index])

def display_movie_details(movie):
    ia.update(movie)
    title = movie.get('title', 'Title not available')
    rating = movie.get('rating', 'Rating not available')
    release_date = movie.get('year', 'Release date not available')
    imdb_id = movie.movieID
    imdb_url = f"https://www.imdb.com/title/tt{imdb_id}/"
    # Display movie details
    result_text.insert(tk.END, f"Title: {title}\n")
    result_text.insert(tk.END, f"Rating: {rating}\n")
    result_text.insert(tk.END, f"Release Date: {release_date}\n")
    result_text.insert(tk.END, f"IMDb URL: {imdb_url}\n")
    display_movie_image(imdb_id)

# Function to display the movie image
def display_movie_image(imdb_id):
    # Fetch additional movie details including the IMDb URL and poster URL
    movie_details = ia.get_movie(imdb_id)
    poster_url = movie_details.get('full-size cover url', '')
    if poster_url:
        # Download and display the movie poster
        image_data = urllib.request.urlopen(poster_url).read()
        image = Image.open(io.BytesIO(image_data))
        image = image.resize((150, 220))
        photo = ImageTk.PhotoImage(image)
        result_text.image_create(tk.END, image=photo)
        result_text.insert(tk.END, "\n")  # Insert a newline after the image
        result_text.image = photo  # Maintain a reference to the image to prevent garbage collection
    else:
        result_text.insert(tk.END, "No poster available for this movie.\n")


In [3]:
def open_imdb_page():
    global current_index
    if recommended_movies:
        movie = recommended_movies[current_index]
        imdb_id = movie.movieID
        imdb_url = f"https://www.imdb.com/title/tt{imdb_id}/"
        webbrowser.open_new(imdb_url)

def open_netflix_page():
    global current_index
    if recommended_movies:
        movie = recommended_movies[current_index]
        # Construct the Netflix URL for the specific movie
        title = movie.get('title', 'Title not available')
        # Replace spaces in the title with '+'
        title_query = '+'.join(title.split())
        netflix_url = f"https://www.netflix.com/search?q={title_query}"
        webbrowser.open_new(netflix_url)

def open_youtube_trailer():
    global current_index
    if recommended_movies:
        movie = recommended_movies[current_index]
        title = movie.get('title', 'Title not available')
        # Placeholder for opening YouTube trailer
        youtube_url = f"https://www.youtube.com/results?search_query={title}+trailer"
        webbrowser.open_new(youtube_url)

def next_movie():
    global current_index
    if recommended_movies:
        current_index = (current_index + 1) % len(recommended_movies)
        clear_results()
        display_movie_details(recommended_movies[current_index])

def clear_results():
    result_text.delete(1.0, tk.END)

In [4]:
root = tk.Tk()
root.title("Movie Recommendation System")
root.geometry("800x600")

welcome_label = ttk.Label(root, text="Welcome to Movie Recommendation System", font=("Helvetica", 16))
welcome_label.pack(pady=10)

keyword_label = ttk.Label(root, text="Enter a movie name:")
keyword_label.pack(pady=(0, 5))

keyword_entry = ttk.Entry(root, width=50)
keyword_entry.pack(pady=5)

search_button = ttk.Button(root, text="Search", command=search_button_click)
search_button.pack(pady=10)

result_text = scrolledtext.ScrolledText(root, width=60, height=15)
result_text.pack(pady=10)

next_button = ttk.Button(root, text="Next", command=next_movie)
next_button.pack(pady=10)

imdb_button = ttk.Button(root, text="Open IMDb", command=open_imdb_page)
imdb_button.pack(side='left', padx=5)

netflix_button = ttk.Button(root, text="Open Netflix", command=open_netflix_page)
netflix_button.pack(side='left', padx=5)

youtube_button = ttk.Button(root, text="Open YouTube Trailer", command=open_youtube_trailer)
youtube_button.pack(side='left', padx=5)

root.mainloop()
